# Imports

In [ ]:
%load_ext autoreload

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import hpc05

import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)
def assert_cluster_checked():
    assert run_cluster.value is True, "Command not run. Check the box above to run."

In [ ]:
import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

### Define constants

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Cluster setup

In [ ]:
run_cluster

In [ ]:
assert_cluster_checked()
hpc05.kill_remote_ipcluster()
run_cluster.value = False

In [ ]:
assert_cluster_checked()
client, dview, lview = hpc05.start_remote_and_connect(200, folder='~/two_dim_majoranas', timeout=180)
run_cluster.value = False

# Define and plot system

In [ ]:
syst_pars = {'Ll' :  500,
               'Lr' : 500,
               'Lm' : 500,
               'Ly' : 15.625,
               'a' :  15.625,
            'mu_from_bottom_of_spin_orbit_bands': True}

_=plotting_results.plot_syst(syst_pars, sns_system.dummy_params)

# Define standard parameters

In [ ]:
params_raw= dict(g_factor = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 1.0,
                 alpha = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = .18,
                 Delta_right = .18,
                 B = 0.5,
                 phase = np.pi/2,
                 T = 0.0)

params = dict(**constants,
              **params_raw)

### Define keys to be varied

In [ ]:
keys_with_bounds = {"phase":[0, 2*np.pi],
                    "B":[0,1.2]}

### Define metrics to be recorded

In [ ]:
metric_params_dict = {
#     "energy_gap":{"tol":1e-5},
                       "pfaffian":{}}

### Define data folder

In [ ]:
data_folder = 'spin_orbit_correction_to_chemical_potential'

# Make ASS

In [ ]:
SOI_ass = ASS(keys_with_bounds,
              syst_pars, params,
              metric_params_dict)

### Add additional dimensions

In [ ]:
alphas = np.linspace(30,60, 10)
mus = params['mu'] + constants['m_eff']/constants['hbar']**2*alphas**2 / 2
print(f'alphas = {alphas}')
print(f'mus = {mus}')

In [ ]:
SOI_ass.add_dimension("mu_from_bottom_of_spin_orbit_bands", [True, False])
SOI_ass.add_dimension("alpha", alphas)

### Make learner

In [ ]:
SOI_ass.make_balancing_learner(1000)
# SOI_ass.load(data_folder)

### Make runner with saver

In [ ]:
runner = adaptive.Runner(SOI_ass.get_balancing_learner())
SOI_ass.start_periodic_saver(runner, data_folder, interval=180)
runner.live_info()

# Plot

In [ ]:
%%opts Image {+framewise} [colorbar=True width=400 height=400]

kdims, plot_dict = SOI_ass.get_plot_dict(200, True)
pdict = dict((k,v[:,:]) for k,v in plot_dict.items())
hv.HoloMap(pdict, kdims=kdims)

In [ ]:
[(l.npoints, l.loss()) for l in runner.learner.learners]


In [ ]:
%%opts Image {+framewise} [colorbar=True width=400 height=400]
SOI_ass.learners[0].plot(tri_alpha=1)

In [ ]:
pc = SOI_ass.learners[3].data.keys()

In [ ]:
px, py = zip(*pc)
hv.Image(np.histogram2d(px, py, 10)[0])

In [ ]:
def density(pointcloud, n=100, gain=100):
    dens = np.zeros((n,n))
    px, py = zip(*pointcloud)
    xdim = np.linspace(min(px), max(px), n)
    ydim = np.linspace(min(py), max(py), n)
    XY = np.meshgrid(xdim, ydim)
    f = lambda xy, xy0: np.exp(-gain*((xy[0]-xy0[0])**2 + (xy[1]-xy0[1])**2))
    for xy0 in pointcloud:
        dens += f(XY, xy0)
    return ((xdim,ydim), dens)

In [ ]:
xy, dens = density(pc, gain=100)
x,y = xy

In [ ]:
hv.Image((x, y, (dens))) #* hv.Points(pc)